# &#128214; Lab 9: Interprocedural Analysis

## &#128214; Exercise 1: Simple Interprocedural Analysis

### &#127919; Objective

Expand your static taint analysis to include interprocedural data flows. Learn to design an analysis that can trace taints across function boundaries. Apply these concepts to complex examples to identify potential sensitive data leaks.

### &#128214; Background

Building upon previous taint analysis knowledge, interprocedural analysis allows us to analyze how data flows across different functions and method calls.

### &#10145; Tasks

#### Task 1: Interprocedural Taint Analysis

**Extend Taint Analysis for Interprocedural Data Flows:**
   - Enhance your taint analysis to track data as it moves from one function to another.

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation
- `abc`: a module to implement abstract methods in Python
- `graphviz`: a library to create directed graphs

In [ ]:
import ast
from abc import ABC, abstractmethod
import graphviz

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
def bridge(data):
    leak_sensitive_data(data)

a = get_password()
bridge(a)


b = get_password()
bridge(b)
"""

### Utility Control Flow Graph class

&#128161; The following cell contains a utility class to build a Control Glow Graph. 
You have to use this class to build the control flow graph, as in previous labs.

In [ ]:
class ControlFlowGraph:
    """
    A class representing a Control Flow Graph (CFG).

    Attributes:
    nodes: A list where each element is a statement.
    edges: A list of tuples representing edges between nodes, where each tuple contains a pair of nodes.

    Methods:
    add_node:
        Adds a new node with the given statement to the graph, returning the new node.
    add_edge:
        Adds an edge between the specified node indices to the graph.
    visualize:
        Prints a visualization of the graph to the console.
    to_dot:
        Returns a DOT-format string representing the graph.
    """

    def __init__(self):
        """
        Initializes a new instance of the ControlFlowGraph class, with empty nodes and edges.
        """
        self.nodes = []
        self.edges = []

    def add_node(self, statement):
        """
        Adds a new node with the given statement to the graph.

        Parameters:
        statement: The statement associated with the new node.

        Returns:
        The node
        """
        self.nodes.append(statement)
        return statement

    def add_edge(self, node1, node2):
        """
        Adds an edge between the specified nodes to the graph.

        Parameters:
        node1: The source node.
        node2: The target node.
        """
        if (node1, node2) not in self.edges:
            self.edges.append((node1, node2))

    def visualize(self):
        """
        Prints a visualization of the graph to the console.
        Each edge is printed as a line in the format 'source -> target'.
        """
        for node in self.nodes:
            node_has_edges = False
            for (source, target) in self.edges:
                if node == source:
                    print(f'{source} -> {target}')
                    node_has_edges = True
            if not node_has_edges:
                print(f'{node}')
                    

    def to_dot(self):
        """
        Returns a DOT-format string representing the graph (for vizualization purposes).

        Returns:
        str: A string in DOT format.
        """
        dot_lines = ['digraph cfg {']
        stmt_to_index = {}
        for index, statement in enumerate(self.nodes):
            stmt_to_index[statement] = index
            dot_lines.append(f'    node{index} [label="{statement}"];')
        for edge in self.edges:
            source, target = edge
            dot_lines.append(f'    node{stmt_to_index[source]} -> node{stmt_to_index[target]};')
        dot_lines.append('}')
        return '\n'.join(dot_lines)

### Function to create the CFG

&#128161; In the following cell, you will use the `build_cfg` function of the previous lab.

In [ ]:
def build_cfg(node, cfg, parent=None):
    """
    Recursively builds a Control Flow Graph (CFG) from an Abstract Syntax Tree (AST) node, considering various types of statements including assignment, expression, conditional, loop, and module level statements.

    Parameters:
    node: The current node in the AST.
    cfg: An instance of ControlFlowGraph to which nodes and edges will be added.
    parent: A list of parent nodes from which edges to the current node will be drawn. 
    
    Returns:
    list: A list of current nodes which will act as parents for the next level of recursion.
    """
    current_nodes = []
    current_node = None
    
    if isinstance(node, ast.Assign):
        current_node = cfg.add_node(node)
        current_nodes.append(current_node)
    elif isinstance(node, ast.Expr) and isinstance(node.value, ast.Call):
        current_node = cfg.add_node(node)
        current_nodes.append(current_node)
    elif isinstance(node, ast.If):
        current_node = cfg.add_node(node)
        if parent is not None:
            for p in parent:
                cfg.add_edge(p, current_node)
        last_body_nodes = [current_node]
        for statement in node.body:
            last_body_nodes = build_cfg(statement, cfg, last_body_nodes)
        last_orelse_nodes = [current_node]
        for statement in node.orelse:
            last_orelse_nodes = build_cfg(statement, cfg, last_orelse_nodes)
        current_nodes.extend(last_body_nodes)
        current_nodes.extend(last_orelse_nodes)
    elif isinstance(node, ast.Module):
        entry_node = cfg.add_node('Entry')
        last_nodes = [entry_node]
        for statement in node.body:
            if last_nodes and len(last_nodes) > 0:
                last_nodes = build_cfg(statement, cfg, last_nodes)
            else:
                last_nodes = build_cfg(statement, cfg, [entry_node])
        return last_nodes
    elif isinstance(node, ast.While):
        current_node = cfg.add_node(node)
        if parent is not None:
            for p in parent:
                cfg.add_edge(p, current_node)
        last_body_nodes = [current_node]
        for statement in node.body:
            last_body_nodes = build_cfg(statement, cfg, last_body_nodes)
        for last_body_node in last_body_nodes:
            cfg.add_edge(last_body_node, current_node)
        current_nodes.extend([current_node])
    elif isinstance(node, ast.For):
        current_node = cfg.add_node(node)
        if parent is not None:
            for p in parent:
                cfg.add_edge(p, current_node)
        last_body_nodes = [current_node]
        for statement in node.body:
            last_body_nodes = build_cfg(statement, cfg, last_body_nodes)
        for last_body_node in last_body_nodes:
            cfg.add_edge(last_body_node, current_node)
        current_nodes.extend([current_node])
        
    if parent is not None:
        for p in parent:
            if current_node is not None:
                cfg.add_edge(p, current_node)
    
    return current_nodes 


### Build the CFG

&#128161; In the following cell, you will build the CFG as in previous labs

### Build the CG

&#128161; In the following cells, you will reuse the code of Lab 7 and build the call graph of the program

In [ ]:
class CallGraph:
    """A class to represent a call graph."""
    
    def __init__(self):
        """Initializes an empty call graph."""
        self.graph = {}
    
    def add_edge(self, caller, callee):
        """
        Adds an edge from caller to callee in the graph.
        
        Parameters:
        caller: The caller function.
        callee: The callee function.
        """
        self.graph.setdefault(caller, set()).add(callee)
    
    def visualize(self):
        """Visualizes the call graph."""
        for caller, callees in self.graph.items():
            for callee in callees:
                print(f'{caller} -> {callee}')
        
                
    def to_dot(self):
        """
        Converts the call graph to a dot representation.
        
        Returns:
        str: The dot representation of the call graph.
        """
        dot_lines = ["digraph CallGraph {"]
        for caller, callees in self.graph.items():
            for callee in callees:
                dot_lines.append(f'    "{caller}" -> "{callee}";')
        dot_lines.append("}")
        return '\n'.join(dot_lines)



In [ ]:
class CallGraphBuilder(ast.NodeVisitor):
    def __init__(self, points_to_set):
        self.call_graph = CallGraph()
        self.points_to_set = points_to_set


In [ ]:
class ClassFunctionAnalyzer(ast.NodeVisitor):
    """
    A node visitor class that walks through the Abstract Syntax Tree (AST) to identify and record all class and function definitions within the code.

    Attributes:
    classes: A set to store unique class names encountered in the AST.
    functions: A set to store unique function names encountered in the AST.
    """
    def __init__(self):
        self.classes = set()
        self.functions = set()

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, adding the class name to the classes set.

        Parameters:
        node: The class definition node being visited.
        """
        self.classes.add(node.name)
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """
        Visits a function definition node, adding the function name to the functions set.

        Parameters:
        node: The function definition node being visited.
        """
        self.functions.add(node.name)
        self.generic_visit(node)
        
    def get_call_type(self, name):
        """
        Determines whether a given name corresponds to a class instantiation or a function call.

        Parameters:
        name: The name of the function or class.

        Returns:
        str: "class" if the name corresponds to a class, "function" if it corresponds to a function, or "unknown" otherwise.
        """
        if name in self.classes:
            return "class"
        elif name in self.functions:
            return "function"
        else:
            return "unknown"

In [ ]:
def build_points_to_set(node, analyzer):
    """
    Analyzes an AST to gather information about potential class instantiations and method calls.
    
    Parameters:
    node: The root node of the AST.
    analyzer: An instance of ClassFunctionAnalyzer.
    
    Returns:
    dict: The points-to set.
    """

### Visualize

&#128161; In the following cell, you will print the call graph with `graphviz`

Before diving into the exercise, you have to understand why building individual CFGs for each function and then integrating them into a unified CFG is an important step in interprocedural analysis.

### Why Build Individual CFGs for Each Function?

1. **Granular Analysis**: Each function in a program can be thought of as an independent unit with its own control flow. By building CFGs for each function, you gain a detailed understanding of the control flow within these individual units.

2. **Foundation for Interprocedural Analysis**: Understanding how control flows within each function is a prerequisite for analyzing how control flows across different functions - which is what interprocedural analysis is all about (otherwise we could not propagate data flow facts across functions).

### Why Integrate into a Unified CFG?

1. **Whole-Program Perspective**: While individual CFGs are great for understanding single functions, real-world programs involve multiple functions calling each other. A unified CFG provides a holistic view of how these function calls interact, which is important for understanding the entire program's behavior and propagate values accordingly.

### Task for the Exercise

With this background, your task is to first build individual CFGs for each function in the given code. Then, integrate these individual CFGs into one unified CFG.
 This unified CG is often called a "super graph" and will serve as a basis for the interprocedural analysis.
 
In the following cell, implement a strategy to build the CFG of all functions in the program, do not forget the root function.
Do not forget that you need to take care of edges. Some edges should disappear and new edges should appear.
Indeed, when there is a call, an edge to the function called CFG should appear, hence the edge to the next statement should disappear.
Also, edges from the end of the callee to the next statement of the caller should appear.

&#128161; Let's focus on building the unified CFG first, setting aside the call graph for the moment to simplify the approach.

&#128161; In the following cell, test your code and display the super graph.

You might see a cycle, this is normal.

Now let us apply the taint analysis on the super graph and see what happens? reuse the code from previous lab

In [ ]:
class DataFlowAnalysis(ABC):
    """
    Abstract Base Class for data flow analysis on a Control Flow Graph (CFG).

    Attributes:
        cfg: Control Flow Graph on which to perform the analysis.
        in_sets: Dictionary to store 'in' sets for each node.
        out_sets: Dictionary to store 'out' sets for each node.
    """

    def __init__(self, cfg):
        """
        Initialize the DataFlowAnalysis class.

        Parameters:
            cfg: Control Flow Graph on which to perform the analysis.
        """

        self.cfg = cfg
        self.in_sets = {node: set() for node in cfg.nodes}
        self.out_sets = {node: set() for node in cfg.nodes}

    def analyze(self):
        """
        Core algorithm for data flow analysis, common to all types of analyses.
        
        Updates the in_sets and out_sets attributes based on the implemented get_gen and get_kill methods.
        """

        first_id = 1
        worklist = [self.cfg.nodes[first_id]]
        visited = set()

        while worklist:
            node = worklist.pop(0)
            visited.add(node)

            # In[node] = Union of Out[predecessors]
            in_set = set().union(
                *[self.out_sets[pred] for pred, succ in self.cfg.edges if succ == node]
            )
            self.in_sets[node] = in_set

            # Out[node] = gen U (In[node] - kill)
            gen = self.get_gen(node)
            kill = self.get_kill(gen, in_set, node)
            out_set = gen.union(in_set - kill)
            self.out_sets[node] = out_set

            successors = {succ for pred, succ in self.cfg.edges if pred == node}
            worklist.extend(succ for succ in successors if succ not in visited)

    @abstractmethod
    def get_gen(self, node):
        """
        Abstract method to get the 'gen' set.
        
        Must be implemented by subclass.

        Returns:
            The 'gen' set.
        """
        pass

    @abstractmethod
    def get_kill(self, gen, in_set, node):
        """
        Abstract method to get the 'kill' set.
        
        Must be implemented by subclass.

        Parameters:
            gen: The 'gen' set.
            in_set: The 'in' set for a node.
            node: the node under analysis

        Returns:
            The 'kill' set.
        """
        pass

In [ ]:
class TaintAnalysis(DataFlowAnalysis, ABC):
    
    def __init__(self, cfg, sources, sinks):
        super().__init__(cfg)
        self.sources = sources
        self.sinks = sinks

&#128161; In the following, apply the taint analysis with `get_password` as a source and `leak_sensitive_data` as a sink.

In [ ]:
sources = set()
sources.add("get_password")
sinks = set()
sinks.add("leak_sensitive_data")
ta = TaintAnalysis(super_graph, sources, sinks)
ta.analyze()
check_leak(super_graph.nodes)

&#128161; It should not work.

Indeed, in the new CFG, only variable `a` holds a tainted value, not `data` yet since the initial context is not the same.
It means we need additional steps to make our taint analysis work.
This is achieved through parameter mapping, there are several ways, from simple to more complex. 
You are free to implement your own way in the following cells, you can update your previous code or produce a new piece of code.
It is not optimal, but for simplicity you are allowed to use variable names for the mapping.

&#128161; In the following cell, you should re-create the super-graph and have a mapping between variables.

To what variable should `data` be mapped to?
In your mapping, to what is `data` mapped to?

&#128161; In the following cells, you will re-run the taint analysis on your super graph.
Do not hesitate to update the previous taint analysis code to take into account your mapping and propagate data flow facts accordingly.

In [ ]:
class TaintAnalysis(DataFlowAnalysis, ABC):
    
    def __init__(self, cfg, sources, sinks, mapping):
        super().__init__(cfg)
        self.sources = sources
        self.sinks = sinks
        self.mapping = mapping

In [ ]:
sources = set()
sources.add("get_password")
sinks = set()
sinks.add("leak_sensitive_data")
ta = TaintAnalysis(super_graph, sources, sinks, mapping)
ta.analyze()
check_leak(super_graph.nodes)

How many leaks does your analysis return?

If the answer is 1, you might want to update the `check_leak` function to take both leaks into account:

Redo the taint analysis:

In [ ]:
sources = set()
sources.add("get_password")
sinks = set()
sinks.add("leak_sensitive_data")
ta = TaintAnalysis(super_graph, sources, sinks, mapping)
ta.analyze()
check_leak(super_graph.nodes)

Good ! But the solution is not perfect, in reality we would not deal with variable names, we would have dedicated objects in memory for each independant variable, etc.

in reality it is more complicated than that, we would have to handle returned values otherwise the taint could not be propagated correctly.

For instance, consider the following example:

```python
def get():
    return get_password()

a = get()
```

In this example, `a` should be tainted, though with our current analysis it will not be tainted since we did not handle return values.

Also, if you test your example with more complicated code, chances are that it does not work anymore since, as already said, in reality code are way more complex and hanlding it statically to model everything would take months, or years.

Our goal is not to develop full-fledge static analyses but to showcase basic stuff.

### &#10067; Questions

Was the call graph useful in our example?

In what case would the call graph be useful?

How would you change the implementation of the interprocedural analysis to use the call graph?

How would you handle the case cited above? How would you handle return values?

&#128161; Propose a modification of your code that would rely on the call graph to connect the dots with calls to methods on objects.

&#128161; In the following cell, you will display, with `graphviz` the latest super graph in memory

Notice the path that is impossible at execution time? 
There is an infeasible path: from the code in `bridge` (second call) to the code that is executed right after the first call to bridge. This is a well-known problem in static analysis, known as a context-sensitivity problem.
Indeed, since we are propagating data flow facts through edges and we are iterating through edges, how could we decide that we need to go back to a particular statement?

Any idea? What would be your strategy? 